task 90
had to rewrite the sh file due to a 404 error I have included my new sh file in with the github


task 91

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, DatasetDict

# Verify installations
try:
    import accelerate
    print(f"Accelerate version: {accelerate.__version__}")
    assert accelerate.__version__ >= '0.26.0', "Accelerate version too old"
except ImportError:
    raise ImportError("Please install accelerate: pip install 'accelerate>=0.26.0'")

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load datasets
try:
    dataset_en = load_dataset('text', data_files={
        'train': 'prep/train.de-en.en',
        'validation': 'prep/valid.de-en.en',
        'test': 'prep/test.de-en.en'
    })
    
    dataset_de = load_dataset('text', data_files={
        'train': 'prep/train.de-en.de',
        'validation': 'prep/valid.de-en.de',
        'test': 'prep/test.de-en.de'
    })
except Exception as e:
    raise FileNotFoundError(f"Error loading dataset files: {e}")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

def preprocess_function(examples):
    # Tokenize inputs and targets together
    model_inputs = tokenizer(
        examples["text"],
        text_target=examples["german_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    return model_inputs

# Prepare datasets
tokenized_datasets = DatasetDict()
for split in ['train', 'validation', 'test']:
    assert len(dataset_en[split]) == len(dataset_de[split]), f"Length mismatch in {split}"
    combined = dataset_en[split].add_column("german_text", dataset_de[split]["text"])
    tokenized_datasets[split] = combined.map(
        preprocess_function,
        batched=True,
        remove_columns=["text", "german_text"]
    )

# Model
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de").to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Changed from eval_strategy to evaluation_strategy
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=2,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    save_strategy="epoch",
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Verify everything works
try:
    print("Starting training...")
    trainer.train()
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("Reduce batch size or use gradient accumulation")
    raise e

/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-04-04 10:26:01.119391: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 10:26:01.172045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743780361.193610  453351 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has alrea

Accelerate version: 1.6.0
Using device: cuda


Map:   0%|          | 0/106704 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/8080 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...


Epoch,Training Loss,Validation Loss
1,0.017500,0.264114
2,0.009000,0.244256
3,0.006500,0.239494


/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


task 92

In [7]:
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/source.spm',
 './results/target.spm',
 './results/added_tokens.json')

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Specify the path to your saved model
model_path = "./results"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Move model to GPU if available
device = 0 if torch.cuda.is_available() else -1
model = model.to(device)

# Translate a sentence
input_text = "Hello world, how are you today?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(input_ids, max_length=40)
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translation)

hallo world, wie geht es dir heute?


task 93

In [20]:
from datasets import load_metric
import numpy as np

import evaluate

bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    # Handle tuple outputs (some models return logits as first element)
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Process labels (-100 is ignored by default in loss computation)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # SacreBLEU requires specific formatting
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]  # Note: List of lists
    
    # Compute metric
    result = evaluate.load("sacrebleu").compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    
    return {"bleu": round(result["score"], 4)}  # Round to 4 decimal places

# Evaluate on test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print("All available metrics:", test_results.keys())  # First check what's available

# The key might be one of these:
bleu_key = next((k for k in test_results.keys() if "bleu" in k.lower()), None)
if bleu_key:
    print(f"Test BLEU score: {test_results[bleu_key]}")
else:
    print("BLEU metric not found. Available metrics:", test_results.keys())


All available metrics: dict_keys(['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'])
BLEU metric not found. Available metrics: dict_keys(['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'])


In [18]:
small_test = tokenized_datasets["test"].select(range(10))
test_results = trainer.evaluate(small_test)
print(test_results)

{'eval_loss': 0.1484411358833313, 'eval_runtime': 0.268, 'eval_samples_per_second': 37.312, 'eval_steps_per_second': 7.462, 'epoch': 3.0}


task 94

In [24]:
# See the first example and all available columns
print("First example:", tokenized_datasets["test"][0])
print("Dataset columns:", tokenized_datasets["test"].column_names)

First example: {'input_ids': [6822, 0, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100], 'attention_mask': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
from tqdm import tqdm

def calculate_bleu_from_tokenized(dataset, tokenizer, model, device="cuda"):
    bleu = evaluate.load("sacrebleu")
    predictions = []
    references = []
    
    for example in tqdm(dataset, desc="Calculating BLEU"):
        # Generate translation
        inputs = {
            "input_ids": torch.tensor([example["input_ids"]]).to(device),
            "attention_mask": torch.tensor([example["attention_mask"]]).to(device)
        }
        outputs = model.generate(**inputs)
        
        # Decode predictions and labels
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        label_text = tokenizer.decode(
            [id for id in example["labels"] if id != -100],  # Remove ignored labels
            skip_special_tokens=True
        )
        
        predictions.append(pred_text.strip())
        references.append([label_text.strip()])  # Note: Must be list of lists
    
    return bleu.compute(predictions=predictions, references=references)["score"]

# Run calculation on a subset first
bleu_score = calculate_bleu_from_tokenized(
    tokenized_datasets["test"].select(range(100)),
    tokenizer,
    model,
    device="cuda" if torch.cuda.is_available() else "cpu"
)
print(f"BLEU score: {bleu_score:.2f}")

Calculating BLEU: 100%|████████████████████████████████████████████████| 100/100 [01:14<00:00,  1.35it/s]

BLEU score: 97.66


task 95

In [27]:
import os

# Create directory if it doesn't exist
os.makedirs("bpe_tokenizer", exist_ok=True)
from tokenizers import ByteLevelBPETokenizer
import os

# Initialize tokenizer
tokenizer = ByteLevelBPETokenizer()

# Create output directory
os.makedirs("bpe_tokenizer", exist_ok=True)

# Train tokenizer
try:
    tokenizer.train(
        files=['prep/train.de-en.en', 'prep/train.de-en.de'],
        vocab_size=30000,
        min_frequency=2,
        special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>",
        ]
    )
    
    # Save tokenizer
    tokenizer.save_model("bpe_tokenizer")
    print("Tokenizer successfully trained and saved!")
    
except Exception as e:
    print(f"Error: {e}")




Tokenizer successfully trained and saved!


task 96

In [28]:
# Modify training arguments to enable logging
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    logging_steps=100,
    report_to="tensorboard",
)

# After training, launch TensorBoard
# In terminal: tensorboard --logdir=./logs

task 97

In [56]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig
from datasets import load_dataset, DatasetDict
from optuna import TrialPruned
from optuna.integration import PyTorchLightningPruningCallback
import optuna
import numpy as np
import sacrebleu  # Add this import

# Initialize tokenizer outside the objective function so it's consistent
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Process labels (-100 is ignored by default in loss computation)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute BLEU score using sacrebleu
    # Note: sacrebleu expects the references to be a list of lists (multiple references per prediction)
    bleu_score = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels]).score
    return {"bleu": bleu_score}

def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_layers = trial.suggest_int("num_layers", 4, 8)

    # Modify model config
    config = AutoConfig.from_pretrained("Helsinki-NLP/opus-mt-en-de")
    config.num_layers = num_layers

    # Initialize model with new config
    model = AutoModelForSeq2SeqLM.from_config(config)
    model.resize_token_embeddings(len(tokenizer))  # Ensure embedding layer matches tokenizer

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=f"./results_trial_{trial.number}",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs',
        logging_steps=100,
        save_total_limit=2,
        predict_with_generate=True,  # Important for seq2seq evaluation
        generation_max_length=128,    # Set appropriate max length
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        greater_is_better=True,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,  # Pass the tokenizer to the trainer
        compute_metrics=compute_metrics,
    )

    # Train and evaluate
    trainer.train()
    eval_result = trainer.evaluate()

    return eval_result["eval_bleu"]

# Run the optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Best hyperparameters
print(study.best_params)
import joblib

joblib.dump(study, "optuna_study.pkl")

[I 2025-04-07 22:58:02,497] A new study created in memory with name: no-name-674fe967-7964-494a-9315-f44a1105f14c
/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_453351/3391345746.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


[W 2025-04-07 22:58:46,501] Trial 0 failed with parameters: {'learning_rate': 1.9053305938687953e-05, 'batch_size': 16, 'num_layers': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_453351/3391345746.py", line 71, in objective
    trainer.train()
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/transformers/trainer.py", line 2245, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/transformers/trainer.py", line 2561, in _inner_training_loop
    and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2025-04-07 22:58:4

KeyboardInterrupt: 

In [54]:
study = joblib.load("optuna_study.pkl")


task 98

In [55]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import sacrebleu
import torch

# Load the model and tokenizer
model_path = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load newstest2014 dataset
newstest2014 = load_dataset("wmt14", "de-en", split="test")

def evaluate_on_dataset(dataset, num_samples=None):
    if num_samples:
        dataset = dataset.select(range(num_samples))
        
    translations = []
    references = []
    
    for example in dataset:
        # Tokenize input
        inputs = tokenizer(example["translation"]["en"], return_tensors="pt", padding=True, truncation=True, max_length=128).to(model.device)
        
        # Generate translation
        outputs = model.generate(**inputs, max_length=128)
        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        translations.append(translation)
        references.append([example["translation"]["de"]])  # sacrebleu expects list of lists
    
    # Calculate BLEU score
    bleu_score = sacrebleu.corpus_bleu(translations, references)
    return bleu_score.score

# Evaluate on newstest2014 (can limit samples for quicker testing)
print("Evaluating on newstest2014...")
bleu_score = evaluate_on_dataset(newstest2014, num_samples=100)  # Remove num_samples for full evaluation
print(f"BLEU score on newstest2014: {bleu_score:.2f}")

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Evaluating on newstest2014...
BLEU score on newstest2014: 9.65


In [63]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

# Load News Commentary dataset
news_commentary = load_dataset("news_commentary", "de-en")

# Preprocess the dataset
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["de"] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_news = news_commentary.map(preprocess_function, batched=True)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./news_commentary_finetuned",
    evaluation_strategy="no",  # Disable evaluation
    save_strategy="steps",      # You can keep saving checkpoints if you want
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=False, # No need to load best if not evaluating
    # metric_for_best_model="loss", # Not needed if not evaluating
    # greater_is_better=False,      # Not needed if not evaluating
)

# Create Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news["train"],
    # eval_dataset=tokenized_news["validation"], # No need to provide eval_dataset
    tokenizer=tokenizer,
)

# Fine-tune on News Commentary
print("Fine-tuning on News Commentary dataset...")
trainer.train()

# Save the adapted model
model.save_pretrained("./news_adapted_model")
tokenizer.save_pretrained("./news_adapted_model")

# Evaluate again on newstest2014
print("Evaluating adapted model on newstest2014...")
adapted_bleu = evaluate_on_dataset(newstest2014, num_samples=100)  # Remove num_samples for full evaluation
print(f"BLEU score after adaptation: {adapted_bleu:.2f} (improvement: {adapted_bleu - bleu_score:.2f})")

/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_453351/3962978051.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Fine-tuning on News Commentary dataset...


Step,Training Loss
100,0.552600
200,0.436900
300,0.429200
400,0.427000
500,0.415000
600,0.419200
700,0.402600
800,0.392500
900,0.393800
1000,0.394800


Evaluating adapted model on newstest2014...
BLEU score after adaptation: 37.99 (improvement: 28.34)


task 99

Device set to use cuda:0


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4322
 * Running on http://172.30.31.243:4322
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/jace/anaconda

SystemExit: 1

/home/jace/anaconda3/envs/rapids-24.06/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
